In [8]:
import pandas as pd
import numpy as np
import pickle
import re

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# mytest의 [3P, TRB, BLK] 컬럼 값으로
# mytrain의 [3P,TRB, BLK] 컬럼 값으로 구성된 공간에서
# 가장 가까운 거리에 있는 선수 5명의 Pos을 추출한 다음,
# 다수결로 mytest 데이터의 Pos를 예상해보자.
# 그리고 예측된 mytest 데이터의 Pos와 실제 mytest 데이터의 Pos를 비교하여
# 정확도, 정밀도, 재현율, F1Score를 조사해보자

# 혹시 시간이 남으면, 선수 수를 변경하여 높은 정확도를 갖는 k값이 얼마인지 조사해보자.

In [2]:
df = pd.read_csv("basketball_stat.csv")
df

,Player,Pos,3P,2P,TRB,AST,STL,BLK
0,Alex Abrines,SG,1.4,0.6,1.3,0.6,0.5,0.1
1,Steven Adams,C,0.0,4.7,7.7,1.1,1.1,1.0
2,Alexis Ajinca,C,0.0,2.3,4.5,0.3,0.5,0.6
3,Chris Andersen,C,0.0,0.8,2.6,0.4,0.4,0.6
4,Will Barton,SG,1.5,3.5,4.3,3.4,0.8,0.5
...,...,...,...,...,...,...,...,...
95,Nikola Vucevic,C,0.3,6.1,10.4,2.8,1.0,1.0
96,Dwyane Wade,SG,0.8,6.2,4.5,3.8,1.4,0.7
97,Dion Waiters,SG,1.8,4.3,3.3,4.3,0.9,0.4
98,Hassan Whiteside,C,0.0,7.0,14.1,0.7,0.7,2.1


In [3]:
mytrain, mytest = train_test_split(df, test_size=0.2)

In [4]:
train_label = mytrain["Pos"]
test_label = mytest["Pos"]

train = mytrain[["3P","TRB","BLK"]]
test = mytest[["3P","TRB","BLK"]]

In [9]:
train_n = MinMaxScaler().fit_transform(train)
test_n = MinMaxScaler().fit_transform(test)

In [15]:
# k = 5 일때 정확도
k=5
neighbor = []
predic_label=[]
for i in range(len(test_n)):
    for j in range(len(train_n)):
        neighbor.append((np.sqrt(sum((test_n[i]-train_n[j])**2)),i,j))
    neighbor = sorted(neighbor)[:k]
    
    cnt = 0
    for n in neighbor:
        if train_label.iloc[n[2]] == "SG":
            cnt+=1
    if cnt>k/2:
        predic_label.append("SG")
    else:
        predic_label.append("C")
# print(predic_label)
table = classification_report(test_label, predic_label)
print(table)

              precision    recall  f1-score   support

           C       0.83      0.50      0.62        10
          SG       0.64      0.90      0.75        10

    accuracy                           0.70        20
   macro avg       0.74      0.70      0.69        20
weighted avg       0.74      0.70      0.69        20



In [16]:
# k = 1~ 15 까지 

k_list = []
for k in range(15):
    neighbor = []
    predic_label=[]
    for i in range(len(test_n)):
        for j in range(len(train_n)):
            neighbor.append((np.sqrt(sum((test_n[i]-train_n[j])**2)),i,j))
        neighbor = sorted(neighbor)[:k]

        cnt = 0
        for n in neighbor:
            if train_label.iloc[n[2]] == "SG":
                cnt+=1
        if cnt>=k/2:
            predic_label.append("SG")
        else:
            predic_label.append("C")
    table = classification_report(test_label, predic_label)
    res = re.search("(accuracy[\s]+)([.\d]+)",table)
    k_list.append([float(res.group(2)),k])

C:\Users\NaEunSu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\NaEunSu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\NaEunSu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
k_list = sorted(k_list, reverse=True)
k_list

[[0.8, 3],
 [0.7, 14],
 [0.7, 13],
 [0.7, 12],
 [0.7, 11],
 [0.7, 10],
 [0.7, 9],
 [0.7, 8],
 [0.7, 7],
 [0.7, 6],
 [0.7, 5],
 [0.7, 4],
 [0.7, 2],
 [0.7, 1],
 [0.5, 0]]

## k = 3일때 정확도가 제일 높게 나왔다.
## 랜덤으로 샘플을 추출하기 때문에 반복할때마다 적당한 k값은 바뀔것이다.